In [6]:
import torch
data = torch.load('brca1_embeddings.pth', map_location='cpu')
index_test = 3892
data[index_test]

/tmp/ipykernel_39105/3545282475.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('brca1_embeddings.pth', map_location='cpu')


tensor([  54.7500, -118.5000,   -1.6172,  ..., -153.0000, -131.0000,
         -30.5000], dtype=torch.bfloat16)

In [9]:
import pandas as pd
brca1 = pd.read_excel("41586_2018_461_MOESM3_ESM.xlsx", header=2)
brca1 = brca1[[
    'chromosome', 'position (hg19)', 'reference', 'alt', 'function.score.mean', 'func.class',
]]

brca1.rename(columns={
    'chromosome': 'chrom',
    'position (hg19)': 'pos',
    'reference': 'ref',
    'alt': 'alt',
    'function.score.mean': 'score',
    'func.class': 'class',
}, inplace=True)
 
brca1['class'] = brca1['class'].replace(['FUNC', 'INT'], 'FUNC/INT')

In [12]:
from Bio import SeqIO
chr17 = str(next(SeqIO.parse("GRCh37.p13_chr17.fna", "fasta")).seq)

In [13]:
WINDOW_SIZE = 128
complement_table = str.maketrans("ATCG", "TAGC")
def parse_sequences2(pos, ref, alt):
    p = pos - 1 
    full_seq = chr17

    ref_seq_start = p - WINDOW_SIZE//2
    ref_seq_end = p + WINDOW_SIZE//2
    ref_seq = full_seq[ref_seq_start:ref_seq_end]
    snv_pos_in_ref = min(WINDOW_SIZE//2, p)
    var_seq = ref_seq[:snv_pos_in_ref] + alt + ref_seq[snv_pos_in_ref+1:]

    ref_seq_rev_comp = ref_seq.translate(complement_table)[::-1]
    var_seq_rev_comp = var_seq.translate(complement_table)[::-1]

    assert len(var_seq) == len(ref_seq)
    assert ref_seq[snv_pos_in_ref] == ref
    assert var_seq[snv_pos_in_ref] == alt

    return ref_seq, var_seq, ref_seq_rev_comp, var_seq_rev_comp

In [14]:
row = brca1.iloc[index_test]
ref_seq, var_seq, ref_seq_rev_comp, var_seq_rev_comp = parse_sequences2(row['pos'], row['ref'], row['alt'])

print(ref_seq)
print(var_seq)
print(ref_seq_rev_comp)
print(var_seq_rev_comp)

TGGGGTCCTGTGGCTCTGTACCTGTGGCTGGCTGCAGTCAGTAGTGGCTGTGGGGGATCTGGGGTATCAGGTAGGTGTCCAGCTCCTGGCACTGGTAGAGTGCTACACTGTCCAACACCCACTCTCGG
TGGGGTCCTGTGGCTCTGTACCTGTGGCTGGCTGCAGTCAGTAGTGGCTGTGGGGGATCTGGGGAATCAGGTAGGTGTCCAGCTCCTGGCACTGGTAGAGTGCTACACTGTCCAACACCCACTCTCGG
CCGAGAGTGGGTGTTGGACAGTGTAGCACTCTACCAGTGCCAGGAGCTGGACACCTACCTGATACCCCAGATCCCCCACAGCCACTACTGACTGCAGCCAGCCACAGGTACAGAGCCACAGGACCCCA
CCGAGAGTGGGTGTTGGACAGTGTAGCACTCTACCAGTGCCAGGAGCTGGACACCTACCTGATTCCCCAGATCCCCCACAGCCACTACTGACTGCAGCCAGCCACAGGTACAGAGCCACAGGACCCCA


In [ ]:
"""
layer_name = 'blocks.28.mlp.l3'
sequences_list = [ref_seq, var_seq, ref_seq_rev_comp, var_seq_rev_comp]
input_ids = torch.tensor(
    [evo2_model.tokenizer.tokenize(s) for s in sequences_list],
    dtype=torch.int,
).to('cuda:0')


_ , embeddings = evo2_model(input_ids, return_embeddings=True, layer_names=[layer_name])
"""

In [16]:
embedding = data[index_test]
print(embedding.shape)

torch.Size([16384])


In [18]:
# Define the model architecture (same as in model1.ipynb)
import torch.nn as nn

class SNVClassifier(nn.Module):
    def __init__(self, input_dim=16384):
        super(SNVClassifier, self).__init__()
        self.network = nn.Sequential(
            # Capa 1
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),

            # Capa 2
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),

            # Capa 3
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),

            # Capa de Salida
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

# Create model instance
model = SNVClassifier(input_dim=16384)

# Load the trained weights (state_dict)
state_dict = torch.load('model_mlp.pth', map_location='cpu')
model.load_state_dict(state_dict)
model.eval()

# Prepare input tensor - convert to float32 and add batch dimension
input_tensor = embedding.to(torch.float32).unsqueeze(0)

# Make prediction
with torch.no_grad():
    prediction = model(input_tensor)
    probability = prediction.item()  # Sigmoid is already applied in the model

print(f"Prediction probability: {probability:.4f}")
print(f"Predicted class: {'LOF' if probability > 0.5 else 'FUNC/INT'}")

# Get the actual class for comparison
actual_class = brca1.iloc[index_test]['class']
print(f"Actual class: {actual_class}")
print(f"Prediction correct: {('LOF' if probability > 0.5 else 'FUNC/INT') == actual_class}")


Prediction probability: 0.0047
Predicted class: FUNC/INT
Actual class: FUNC/INT
Prediction correct: True


/tmp/ipykernel_39105/1958521554.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('model_mlp.pth', map_location='cpu')


In [21]:
torch.save(embedding, 'sample_patient.pth')